In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
import sys
import re 
import numpy as np 
import pandas as pd
import music21
from glob import glob
import IPython
from tqdm import tqdm
import pickle
from keras.utils import np_utils
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from music21 import converter, instrument, note, chord, stream

In [5]:
songs = glob('/content/drive/My Drive/Final Project/10MinSaxo/*.mid')
print(len(songs))

3


In [0]:
def get_notes():
    notes = []
    for file in songs:
        midi = converter.parse(file)
        notes_to_parse = []
        
        try:
            parts = instrument.partitionByInstrument(midi)
        except:
            pass
        if parts:
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi.flat.notes
    
        for element in notes_to_parse: 
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif(isinstance(element, chord.Chord)):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    with open('/content/drive/My Drive/Final Project/data/notes_3', 'wb') as filepath:
        pickle.dump(notes, filepath)
    
    return notes

In [0]:
songs = songs[:3]

In [0]:
def prepare_sequences(notes, n_vocab): 
    sequence_length = 100

    pitchnames = sorted(set(item for item in notes))
    print('Pitchnames: ',len(pitchnames))

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i: i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    
    n_patterns = len(network_input)
    
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    network_input = network_input / float(n_vocab)
    
    network_output = np_utils.to_categorical(network_output)
    
    return (network_input, network_output)

In [0]:
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout, Flatten

def create_network(network_in, n_vocab): 

    model = Sequential()
    model.add(LSTM(128, input_shape=network_in.shape[1:], return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

In [0]:
from keras.callbacks import ModelCheckpoint

def train(model, network_input, network_output, epochs): 
    # Save model weights.
    filepath = '/content/drive/My Drive/Final Project/Weights/weights.best.music6.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
    
    history = model.fit(network_input, network_output, epochs=epochs, batch_size=32, callbacks=[checkpoint])
    
    plt.plot(history.history['loss'])
    #plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [0]:
def train_network():
    epochs = 50
    
    notes = get_notes()
    print('Notes processed')
    
    n_vocab = len(set(notes))
    print('Vocab generated',n_vocab)
    
    network_in, network_out = prepare_sequences(notes, n_vocab)
    print('Input and Output processed')
    
    model = create_network(network_in, n_vocab)
    print('Model created')
    print('Training in progress')
    train(model, network_in, network_out, epochs)
    print('Training completed')

In [0]:
train_network()

Notes processed
Vocab generated 26
Pitchnames:  26
Input and Output processed
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model created
Training in progress
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
2518/2518 [==============================] - 24s 9ms/step - loss: 2.4600
Epoch 2/50
2518/2518 [==============================] - 23s 9ms/step - loss: 2.2264
Epoch 3/50
2518/2518 [==============================] - 23s 9ms/step - loss: 2.0846
Epoch 4/50
2400/2518 [===========================>..] - ETA: 1s - loss: 1.9924

In [0]:
def generate():
  
    with open('/content/drive/My Drive/Final Project/data/notes_1', 'rb') as filepath:
        notes = pickle.load(filepath)

    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(set(notes))
    
    print('Initiating music generation process.......')
    
    network_input = get_inputSequences(notes, pitchnames, n_vocab)
    
    sequence_length = 100
    
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    network_input = network_input / float(n_vocab)
    
    model = create_network(network_input, n_vocab)
    print('Loading Model weights.....')
    model.load_weights('/content/drive/My Drive/Final Project/Weights/weights.best.music5.hdf5')
    print('Model Loaded')
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [0]:
def get_inputSequences(notes, pitchnames, n_vocab):
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])

    return (network_input)

In [0]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    pattern = network_input[start]
    prediction_output = []
    
    print('Generating notes........')

    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        
        prediction_output.append(result)
        
        pattern = np.concatenate((pattern, [[index]]))
        pattern = pattern[1:len(pattern)]

    print('Notes are generated...')
    return prediction_output

In [0]:
def create_midi(prediction_output):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    
    print('Saving Output file as midi....')

    midi_stream.write('midi', fp='/content/drive/My Drive/Final Project/test_output_10.mid')

In [32]:
generate()

Initiating music generation process.......
Loading Model weights.....
Model Loaded
Generating notes........
Notes are generated...
Saving Output file as midi....
